In [42]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error

In [43]:
df1 = pd.read_csv("Data/PreProcessedData.csv")
df_no_plot = df1.drop('Plot', axis = 1)
df2 = pd.read_csv("Data/LDA_topics.csv") #UPDATE THIS TO BE WHATEVER DATA WE GIVE IT
df_LDATOPICS = pd.concat([df_no_plot, df2], axis=1)
df_LDATOPICS = df_LDATOPICS.drop('Title', axis = 1)
print(df_LDATOPICS)

      IMDbRating  Year  Action  Adult  Adventure  Animation  Biography  \
0            7.1  2000       0      0          0          0          0   
1            4.1  2000       0      0          0          0          0   
2            6.6  2000       0      0          1          0          0   
3            5.6  2000       0      0          0          0          0   
4            7.7  2000       0      0          0          0          0   
...          ...   ...     ...    ...        ...        ...        ...   
6509         4.0  2022       0      0          0          0          0   
6510         7.9  2022       0      0          1          1          0   
6511         7.6  2022       0      0          0          0          0   
6512         6.9  2022       0      0          0          0          1   
6513         6.7  2022       0      0          0          0          0   

      Comedy  Crime  Documentary  ...  Unnamed: 0        t1        t2  \
0          0      0            0  ... 

In [44]:
df2 = pd.read_csv("Data/LDA_topics_synonym.csv")
df_LDATOPICS_synonym = pd.concat([df_no_plot, df2], axis=1)
df_LDATOPICS_synonym = df_LDATOPICS_synonym.drop('Title', axis = 1)

In [45]:
def test_trainsets(df):
    rating = df['IMDbRating']
    xdf = df.drop('IMDbRating', axis=1, inplace=False)
    
    # Split the data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(xdf, rating, test_size=0.2, random_state=42)
    
    X_train_df = pd.DataFrame(X_train)
    X_test_df = pd.DataFrame(X_test)
    y_train_df = pd.DataFrame(y_train)
    y_test_df = pd.DataFrame(y_test)

    return X_train_df, y_train_df, X_test_df, y_test_df

In [46]:
#ROOM FOR PARAMETER TESTING

In [47]:
X_train, y_train, X_test, y_test = test_trainsets(df_LDATOPICS)

# Create the XGBoost model
xgb_model = xgb.XGBRegressor()

# Kfold
kf = KFold(n_splits=5, shuffle=True, random_state=123)

# store the score for each fold
scores = []

mae_scores = []
baseline_scores = []
# Iterate over the folds
for train_index, val_index in kf.split(X_train):
    # Split the data into training and validation sets
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

    # Train the model on the training data
    xgb_model.fit(X_train_fold, y_train_fold)

    # Make predictions on the validation data
    y_pred_fold = xgb_model.predict(X_val_fold)
    # Compute the MAE score
    baseline = mean_absolute_error(y_val_fold, np.array([np.mean(y_train['IMDbRating'])] * len(y_pred_fold)))
    mae = mean_absolute_error(y_val_fold, y_pred_fold)
    print(f"MAE: {mae}")
    print(f"Baseline: {baseline}")
    mae_scores.append(mae)
    baseline_scores.append(baseline)

# Compute the average MAE score
average_mae_LDA = sum(mae_scores) / len(mae_scores)
average_baseline = sum(baseline_scores) / len(baseline_scores)
print(f"Average MAE over training: {average_mae_LDA}")
print(f"Average Baseline over training: {average_baseline}")
y_pred = xgb_model.predict(X_test)
test_score_LDA = mean_absolute_error(y_pred, y_test)
results = pd.DataFrame({'Actual': y_test['IMDbRating'], 'Prediction': y_pred})
results.to_csv('LDA_Topics_results.csv', index=False)

MAE: 0.678209176159545
Baseline: 0.776840421462601
MAE: 0.732682787022069
Baseline: 0.8239868527045442
MAE: 0.7345483812855668
Baseline: 0.8302755760643642
MAE: 0.6901474795277426
Baseline: 0.7796659473113684
MAE: 0.7152102596700306
Baseline: 0.8062892206100271
Average MAE over training: 0.7101596167329908
Average Baseline over training: 0.803411603630581


In [48]:
X_train, y_train, X_test, y_test = test_trainsets(df_LDATOPICS_synonym)

# Create the XGBoost model
xgb_model = xgb.XGBRegressor()

# Kfold
kf = KFold(n_splits=5, shuffle=True, random_state=123)

# store the score for each fold
scores = []

mae_scores = []
baseline_scores = []
# Iterate over the folds
for train_index, val_index in kf.split(X_train):
    # Split the data into training and validation sets
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

    # Train the model on the training data
    xgb_model.fit(X_train_fold, y_train_fold)

    # Make predictions on the validation data
    y_pred_fold = xgb_model.predict(X_val_fold)
    # Compute the MAE score
    baseline = mean_absolute_error(y_val_fold, np.array([np.mean(y_train['IMDbRating'])] * len(y_pred_fold)))
    mae = mean_absolute_error(y_val_fold, y_pred_fold)
    print(f"MAE: {mae}")
    print(f"Baseline: {baseline}")
    mae_scores.append(mae)
    baseline_scores.append(baseline)

# Compute the average MAE score
average_mae_LDASynonym = sum(mae_scores) / len(mae_scores)
average_baseline = sum(baseline_scores) / len(baseline_scores)

print(f"Average MAE over training: {average_mae_LDASynonym}")
print(f"Average Baseline over training: {average_baseline}")
y_pred = xgb_model.predict(X_test)
test_score_LDA_synonym = mean_absolute_error(y_pred, y_test)
results = pd.DataFrame({'Actual': y_test['IMDbRating'], 'Prediction': y_pred})
results.to_csv('LDA_Topics_synonym_results.csv', index=False)

MAE: 0.6905952620026249
Baseline: 0.776840421462601
MAE: 0.7194052845868862
Baseline: 0.8239868527045442
MAE: 0.7220759663609305
Baseline: 0.8302755760643642
MAE: 0.6922737716484436
Baseline: 0.7796659473113684
MAE: 0.7190202249446437
Baseline: 0.8062892206100271
Average MAE over training: 0.7086741019087058
Average Baseline over training: 0.803411603630581


In [49]:
df_HFTransformer = pd.read_csv("Data/PreProcessedData_with_HF_embeddings.csv")
X_train, y_train, X_test, y_test = test_trainsets(df_HFTransformer)

# Create the XGBoost model
xgb_model = xgb.XGBRegressor()

# Kfold
kf = KFold(n_splits=5, shuffle=True, random_state=123)

# store the score for each fold
scores = []

mae_scores = []
baseline_scores = []
# Iterate over the folds
for train_index, val_index in kf.split(X_train):
    # Split the data into training and validation sets
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

    # Train the model on the training data
    xgb_model.fit(X_train_fold, y_train_fold)

    # Make predictions on the validation data
    y_pred_fold = xgb_model.predict(X_val_fold)
    # Compute the MAE score
    baseline = mean_absolute_error(y_val_fold, np.array([np.mean(y_train['IMDbRating'])] * len(y_pred_fold)))
    mae = mean_absolute_error(y_val_fold, y_pred_fold)
    print(f"MAE: {mae}")
    print(f"Baseline: {baseline}")
    mae_scores.append(mae)
    baseline_scores.append(baseline)

# Compute the average MAE score
average_mae_HF = sum(mae_scores) / len(mae_scores)
average_baseline = sum(baseline_scores) / len(baseline_scores)

print(f"Average MAE over training: {average_mae_HF}")
print(f"Average Baseline over training: {average_baseline}")
y_pred = xgb_model.predict(X_test)
test_score_HF = mean_absolute_error(y_pred, y_test)
results = pd.DataFrame({'Actual': y_test['IMDbRating'], 'Prediction': y_pred})

results.to_csv('HF_Transformer_Model_Results.csv', index=False)

MAE: 0.7065315148745835
Baseline: 0.776840421462601
MAE: 0.7376345328772137
Baseline: 0.8239868527045442
MAE: 0.7452298975265415
Baseline: 0.8302755760643642
MAE: 0.7045498759183683
Baseline: 0.7796659473113684
MAE: 0.7379685519981749
Baseline: 0.8062892206100271
Average MAE over training: 0.7263828746389762
Average Baseline over training: 0.803411603630581


In [50]:
print("TRAINING AVERAGE MAE")
print(f"HuggingFaceTransformer: {average_mae_HF}, LDA: {average_mae_LDA}, LDA with synonyms: {average_mae_LDASynonym}, Baseline: {average_baseline}")
print("TEST MAE")
print(f"HuggingFaceTransformer: {test_score_HF}, LDA: {test_score_LDA}, LDA with synonyms: {test_score_LDA_synonym}, Baseline: {mean_absolute_error(y_test, np.array([np.mean(y_train['IMDbRating'])] * len(y_test)))}")


TRAINING AVERAGE MAE
HuggingFaceTransformer: 0.7263828746389762, LDA: 0.7101596167329908, LDA with synonyms: 0.7086741019087058, Baseline: 0.803411603630581
TEST MAE
HuggingFaceTransformer: 0.7603872038269629, LDA: 0.7250767980825874, LDA with synonyms: 0.7157429137050603, Baseline: 0.8261005226413869
